In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import imageio
import os
from PIL import Image

In [ ]:
ds_name = "kidney_3_dense"
train_dir = "/kaggle/input/blood-vessel-segmentation/train/"
msks_dir = f"{train_dir}kidney_3_dense/"
imgs_dir = f"{train_dir}kidney_3_sparse/"
slices_ids = pd.Series(os.listdir(f"{msks_dir}/labels")).sort_values().reset_index(drop=True)
slices_ids

In [ ]:
# Define the function to update the plot
def update_plot(idx):
    if idx % 100 == 0: print(f"Slice {idx + 1} of {len(slices_ids)}")
    # Clear the current plot
    ax[0].clear()
    ax[1].clear()

    # Load the image and label
    img = Image.open(imgs_dir + "images/" + slices_ids[idx])
    msk = Image.open(msks_dir + "labels/" + slices_ids[idx])

    # Display the image and label
    ax[0].imshow(img)
    ax[1].imshow(msk)
    fig.suptitle(f"{ds_name} Slice {idx + 1} of {len(slices_ids)}")
    plt.tight_layout()

# Create the figure and axes
fig, ax = plt.subplots(1, 2, figsize=(15, 11))

# Create the animation
anim = animation.FuncAnimation(fig, update_plot, frames=len(slices_ids), interval=100)
anim.save(f'{ds_name}.mp4', writer='ffmpeg')